## 图数据库同步

In [8]:
from py2neo import Graph, Node, Relationship
from datetime import datetime
import requests

# 连接到 Neo4j 数据库
graph = Graph("bolt://localhost:7687", user="neo4j", password="123456")

# 读取数据文件并解析数据
data_file = "command_data.test"  # 请根据实际文件路径修改

def get_class_probability(command):
    url = 'http://localhost:5000/predict'  # 请根据实际情况修改 URL
    headers = {'Content-Type': 'application/json'}
    data = {'new_command': command}

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        response_data = response.json()
        class_probabilities = response_data.get("Class Probabilities", [[]])[0]
        if class_probabilities:
            return class_probabilities[1]  # 取第二个值
    return None

def get_anomaly_score(timestamp, ppcomm, pcomm):
    url = "http://localhost:5001/anomaly_score"  # 替换成你的Flask应用的URL

    data = {
        "timestamp": timestamp,
        "ppcomm": ppcomm,
        "pcomm": pcomm
    }

    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            result = response.json()
            return result.get('anomaly_score')
        else:
            print(f"Request failed with status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None


with open(data_file, 'r') as file:
    for line in file:
        data = line.strip().split('"')
        #print(data)
        uid, time, child_comm, child_pid, parent_comm, parent_pid, return_code, cmdline = data[1], data[3], data[5], data[7], data[9], data[11], data[13], data[15]
        #print(timestamp, child_comm, child_pid, parent_comm, parent_pid, return_code, cmdline)
        date_object = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
        timestamp = date_object.timestamp()
        # 创建父进程和子进程节点
        parent_process = Node("Process", pid=parent_pid, name=parent_comm, comm=parent_comm)
        child_process = Node("Process", pid=child_pid, name=child_comm, comm=child_comm)

        # 使用 Relationship() 创建关系
        rel = Relationship(parent_process, "create_process", child_process)
        rel["uid"] = uid
        rel["time"] = timestamp
        rel["cmdline"] = cmdline
        rel["return"] = return_code
        #rel["evil_weight"] = get_class_probability(cmdline)
        #rel["anomaly_score"] = get_anomaly_score(timestamp,parent_comm,child_comm)


        # 使用 graph.merge 来合并节点和关系，确保它们在数据库中是唯一的
        graph.merge(parent_process, "Process", "pid")
        graph.merge(child_process, "Process", "pid")
        graph.merge(rel)

print("Data imported into Neo4j database.")


IndexError: list index out of range

## 行为窗口切分

In [6]:
from py2neo import Graph

# 建立与Neo4j数据库的连接
graph = Graph("bolt://localhost:7687", user="neo4j", password="123456")

# 第一个查询，获取所有不重复的communityId
query1 = """
CALL gds.louvain.stream('myGraph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN DISTINCT communityId
ORDER BY communityId ASC
"""

result1 = graph.run(query1).data()

# 存储所有不重复的communityId
community_ids = [record['communityId'] for record in result1]

# 存储每个community的结果
community_results = []

# 第二个查询，对每个communityId执行
for community_id in community_ids:
    query2 = f"""
    CALL gds.louvain.stream('myGraph')
    YIELD nodeId, communityId, intermediateCommunityIds
    WHERE communityId = {community_id}
    MATCH (n)-[r]->(m)
    WHERE id(n) = nodeId
    RETURN n, r, m
    ORDER BY communityId ASC
    """
    result2 = graph.run(query2).data()
    community_results.append(result2)

# 打印结果
for i, community_id in enumerate(community_ids):
    print("****"*10)
    print(f"Community {community_id}:")
    for record in community_results[i]:
        print("----"*10)
        node = record['n']
        relationship = record['r']
        related_node = record['m']
        print(f"PPCOMM: {node['name']}")
        print(f"Rel: {relationship}")
        print(f"PCOMM: {related_node['name']}")



****************************************
Community 1:
----------------------------------------
PPCOMM: bash
Rel: (bash)-[:create_process {cmdline: 'touch /tmp/test', return: '0', time: 1698565353.0, uid: '0'}]->(touch)
PCOMM: touch
----------------------------------------
PPCOMM: bash
Rel: (bash)-[:create_process {cmdline: 'docker exec -it a0 bash', return: '0', time: 1698565306.0, uid: '0'}]->(docker)
PCOMM: docker
----------------------------------------
PPCOMM: bash
Rel: (bash)-[:create_process {cmdline: 'ping baidu.com', return: '0', time: 1698565301.0, uid: '0'}]->(ping)
PCOMM: ping
----------------------------------------
PPCOMM: bash
Rel: (bash)-[:create_process {cmdline: 'docker ps', return: '0', time: 1698565292.0, uid: '0'}]->(docker)
PCOMM: docker
----------------------------------------
PPCOMM: bash
Rel: (bash)-[:create_process {cmdline: 'ifconfig', return: '0', time: 1698565280.0, uid: '0'}]->(ifconfig)
PCOMM: ifconfig
----------------------------------------
PPCOMM: bash
